**Pirate Intelligent Agent: Q-Learning Maze Navigation**


## Overview

This Jupyter Notebook presents a reinforcement learning (RL) project developed for the CS 370 course. 
The project implements a Q-learning-based intelligent pirate agent to navigate a treasure maze. The main goals are:

- Train an AI agent to find the treasure.
- Avoid obstacles in the maze.
- Learn an optimal path using the Q-learning algorithm.


In [ ]:

import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=2)


In [ ]:

class TreasureMaze:
    def __init__(self, size=5):
        self.size = size
        self.state_space = [(i, j) for i in range(size) for j in range(size)]
        self.action_space = ['up', 'down', 'left', 'right']
        self.reset()

    def reset(self):
        self.agent_pos = [0, 0]
        self.treasure_pos = [self.size - 1, self.size - 1]
        self.obstacles = [[1, 1], [2, 3], [3, 1]]
        return self.get_state()

    def get_state(self):
        return tuple(self.agent_pos)

    def is_terminal(self):
        return self.agent_pos == self.treasure_pos

    def move(self, action):
        old_pos = self.agent_pos.copy()

        if action == 'up':
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 'down':
            self.agent_pos[0] = min(self.size - 1, self.agent_pos[0] + 1)
        elif action == 'left':
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        elif action == 'right':
            self.agent_pos[1] = min(self.size - 1, self.agent_pos[1] + 1)

        if self.agent_pos in self.obstacles:
            self.agent_pos = old_pos  # Hit an obstacle, stay in place

        reward = -1
        if self.agent_pos == self.treasure_pos:
            reward = 10

        return self.get_state(), reward, self.is_terminal()


In [ ]:

def qtrain(env, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.2):
    q_table = {}

    for state in env.state_space:
        q_table[state] = {a: 0 for a in env.action_space}

    for ep in range(episodes):
        state = env.reset()
        done = False

        while not done:
            if random.random() < epsilon:
                action = random.choice(env.action_space)
            else:
                action = max(q_table[state], key=q_table[state].get)

            next_state, reward, done = env.move(action)
            old_q = q_table[state][action]
            next_max_q = max(q_table[next_state].values())

            q_table[state][action] = old_q + alpha * (reward + gamma * next_max_q - old_q)
            state = next_state

    return q_table


In [ ]:

env = TreasureMaze()
q_table = qtrain(env)

# Display sample Q-values
for state in sorted(q_table.keys()):
    print(f"State {state}: {q_table[state]}")
